In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample
from collections import Counter
import string

In [2]:
def readTxt(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep='\t')
    
    return df

In [3]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep=',')
    
    return df

In [59]:
def toCsvDf(fileName, df):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df.to_csv(fullFileName, header = False, sep='\t', index = False)

In [58]:
def toCsvNp(fileName, npArray):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')

In [60]:
def createDictionary():
    allWords = list()
    cnt = Counter()
    trainingDF = readTxt('\IMDB-train.txt')
    translator = str.maketrans('','', string.punctuation)
    
    for i in range(0, len(trainingDF)):
        allWords.extend(trainingDF.iloc[i,0].translate(translator).lower().split(" "))
    
    for word in allWords:
        cnt[word] +=1

    
    
    dictionaryWords = list(zip(*cnt.most_common(10000)))[0]
    
    dictionary = {}
    for index, key in enumerate(dictionaryWords):
        dictionary[key] = index
#     print(dictionary)
    
    newArray = np.asarray(cnt.most_common(10000))
    withIndexVocabArray = np.insert(newArray, 1, range(0,10000),1)
    toCsvDf('\IMDB-vocab.txt', pd.DataFrame(withIndexVocabArray))

    return dictionary
# createDictionary()

In [76]:
def convertWordsToVector(trainingRow, dictionary):
    translator = str.maketrans('','', string.punctuation)
    returnRow = trainingRow.translate(translator).lower().split(" ")
    vector = np.zeros(10000, dtype=np.int8)
    
    for word in returnRow:
        if word in dictionary:
            vector[dictionary[word]] = np.int8(1)
            
    return vector

In [90]:
# TODO: CHANGE DICTIONARY CREATION FROM ONLY IMDB TRAINING TO YELP TRAINING AS WELL
def createBagOfWordsMatrix(df):
    dictionary = createDictionary()    
#     trainingDF = readTxt('\IMDB-train.txt')
    translator = str.maketrans('','', string.punctuation)
    
    distancesArray = np.zeros((len(df), len(dictionary)), dtype=np.int8)
        
    for i in range(0, len(df)):
        vector = convertWordsToVector(df.iloc[i,0], dictionary)
        distancesArray[i] = vector

    return distancesArray
trainingDF = readTxt('\IMDB-train.txt')
npArray = createBagOfWordsMatrix(trainingDF)
print(npArray)
# dfArray = pd.DataFrame(npArray)
# dfArray.to_csv(r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets\TESTING2.csv', header = False, sep=',', index = False)

[[1 1 0 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 ..., 
 [1 0 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]
 [1 1 1 ..., 0 0 0]]


In [93]:
def main():
    trainingDF = readTxt('\IMDB-train.txt')
    validationDF = readTxt('\IMDB-valid.txt')
    from sklearn.dummy import DummyClassifier
    from numpy.random import random_integers
    randomClassifier = DummyClassifier(strategy='uniform')
    randomClassifier.fit(createBagOfWordsMatrix(trainingDF), trainingDF[1])
    print(randomClassifier.predict(createBagOfWordsMatrix(validationDF)))
    
    
main()

[1 1 1 ..., 1 1 0]
